<a href="https://colab.research.google.com/github/JonathanFly/3d-ken-burns/blob/master/3D_Ken_Burns_HighResolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic 3D Ken Burns Effect from a Single Image
### Updated version with Full Rez & FPS selection

A PyTorch based implementation of the techniques presented in the following paper: ['3D Ken Burns Effect from a Single Image'](https://arxiv.org/abs/1909.05483).

## Note
Before running this notebook make sure that your runtime type is 'Python 3 with GPU acceleration'.

## Credits
- Original Implementation by: [Simon Niklaus](https://github.com/sniklaus/3d-ken-burns)
- Updated Implementation by: [Preston Allen - pressreset](https://github.com/pressreset/Multi-FPS-Full-Rez-3d-ken-burns)
- Original Google Colab notebook by: [Arnaldo Gabriel](https://github.com/agmm/colab-3d-ken-burns)
- Updated Google Colab by: [Preston Allen - pressreset](https://github.com/pressreset/Colab-Multi-FPS-Full-Rez-3d-Ken-Burns)
- A few bug fixes to Colab by [https://twitter.com/jonathanfly](https://twitter.com/jonathanfly)

## More Info
- Paper: https://arxiv.org/abs/1909.05483
- Repo: https://github.com/sniklaus/3d-ken-burns
- Video: https://www.youtube.com/watch?v=WrajxHHfRBA

In [0]:
# Setup Everything. This takes quite awhile.
!git clone https://github.com/JonathanFly/3d-ken-burns.git
# Move into the downloaded repository
%cd /content/3d-ken-burns
# Install dependencies
!pip install cupy moviepy
# Setup environment variable for CUDA
%env CUDA_HOME=/usr/local/cuda
# Download the pre-trained models
!bash download.bash

In [0]:
#Testing High Rez ouput, skip this cell unless you want to make sure everything is working.
!wget https://wallpaperaccess.com/full/1267.jpg --output-document ./images/TokyoHighRez.jpg
!python fullrez.py --fps 30 --bitrate "18M" --in ./images/TokyoHighRez.jpg --out ./videos/TokyoHighRez.mp4



---

#### fullrez.py
Process single file at full rez.

fullrez.py will process the image at full resolution. You can process HD (1920x1080) and 2k (3840x2160) images on a 16GB card, and possibly a 11GB card. 4k is out of the question

> Options are:

*   --fps (integer value) | default: 30
*   --bitrate (string value) | default: "12M"
*   --in (/directory path/filename.[ jpg | png ]) | default: ./images/doublestrike.jpg
*   --out (/directory path/filename.mp4) | default: ./videos/fullrez.mp4

> Examples:

> !python fullrez.py --fps 60 --in ./images/myimage.png

> !python fullrez.py --fps 30 --bitrate "18M" --in ./images/doublestrike.jpg --out ./videos/doublestrike.mp4




In [0]:
# Generate fullrez video
!python fullrez.py --fps 30 --bitrate "18M" --in ./images/doublestrike.jpg --out ./videos/doublestrike.mp4

# Changing Zoom targets and levels

You can customize `objectFrom` and `objectTo` here. 

`'dblZoom': 1.25` is the zoom level but for very large zoom you will also need to increase:

`'dblSteps': numpy.linspace(0.0, 1.0, 75)`
change to:
`'dblSteps': numpy.linspace(0.0, 10.0, 75)`

(10.0 is just an example.)

You might also want to try changing the `75` a larger number will slow down the camera movement. For example:

`'dblSteps': numpy.linspace(0.0, 10.0, 275)`

In general the more you increase the zoom way past what this designed for the more you want to turn down `'dblShift': 100.0` so if you zoom far try `10.0` instead of `100.0`. The camera will fly way off to the side if you don't. You can get some angles by changing the center points but it often results in a crash.

To update fullrez.py click the play button on the cell below after making your changes. Every time you make another change you have to click the cell play button again to update the file.

If you are looking for more things to tweak take a look at common.py.

In [0]:
%%writefile /content/3d-ken-burns/fullrez.py
#You can edit this cell then click the play button to update the file. 
#!/usr/bin/env python

import torch
import torchvision

import base64
import cupy
import cv2
import flask
import getopt
import gevent
import gevent.pywsgi
import h5py
import io
import math
import moviepy
import moviepy.editor
import numpy
import os
import random
import re
import scipy
import scipy.io
import shutil
import sys
import tempfile
import time
import urllib
import zipfile

##########################################################

assert(int(str('').join(torch.__version__.split('.')[0:3])) >= 120) # requires at least pytorch version 1.2.0

torch.set_grad_enabled(False) # make sure to not compute gradients for computational performance

torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

##########################################################

objectCommon = {}

exec(open('./common.py', 'r').read())

exec(open('./models/disparity-estimation.py', 'r').read())
exec(open('./models/disparity-adjustment.py', 'r').read())
exec(open('./models/disparity-refinement.py', 'r').read())
exec(open('./models/pointcloud-inpainting.py', 'r').read())

##########################################################

arguments_strIn = './images/doublestrike.jpg'
arguments_strOut = './videos/fullrez.mp4'
arguments_strFps = '30'
arguments_strBitRate = "12M"

for strOption, strArgument in getopt.getopt(sys.argv[1:], '', [ strParameter[2:] + '=' for strParameter in sys.argv[1::2] ])[0]:
	if strOption == '--fps' and strArgument != '': arguments_strFps = strArgument # allow FPS value selection from cmd line
	if strOption == '--bitrate' and strArgument != '': arguments_strBitRate = strArgument # allow FPS value selection from cmd line
	if strOption == '--in' and strArgument != '': arguments_strIn = strArgument # path to the input image
	if strOption == '--out' and strArgument != '': arguments_strOut = strArgument # path to where the output should be stored
	# end

	##########################################################
if __name__ == '__main__':
	numpyImage = cv2.imread(filename=arguments_strIn, flags=cv2.IMREAD_COLOR)

	intWidth = numpyImage.shape[1]
	intHeight = numpyImage.shape[0]

	dblRatio = float(intWidth) / float(intHeight)

	process_load(numpyImage, {})

	objectFrom = {
	'dblCenterU': intWidth / 2.0, #You can change these but many settings will crash it
	'dblCenterV': intHeight / 2.0, #You can change these but many settings will crash it
	'intCropWidth': int(math.floor(0.98 * intWidth)),
	'intCropHeight': int(math.floor(0.98 * intHeight))
	}

	objectTo = process_autozoom({
	#'dblCenterU': intWidth / 2.0, #Not sure of effect
	#'dblCenterV': intHeight / 2.0, #Not sure of effect
	'dblShift': 100.0, #original
	#'dblShift': 10.0, #try this with large zooms
	#'dblZoom': 40.25, #x50 zoom
	'dblZoom': 1.25, #original
	'objectFrom': objectFrom
	})

	numpyResult = process_kenburns({
	#'dblSteps': numpy.linspace(0.0, 40.0, 800).tolist(), #example very large zoom, slow. May run out of memory and crash.
	'dblSteps': numpy.linspace(0.0, 1.0, 75).tolist(), #original settings
	#'dblSteps': numpy.linspace(0.0, 20.0, 275).tolist(), # Zoom x20 and more frames 
	'objectFrom': objectFrom,
	'objectTo': objectTo,
	'boolInpaint': True
	})

	moviepy.editor.ImageSequenceClip(sequence=[ numpyFrame[:, :, ::-1] for numpyFrame in numpyResult + list(reversed(numpyResult))[1:] ], fps=float(arguments_strFps)).write_videofile(arguments_strOut, bitrate=arguments_strBitRate)
	# end


# Video Won't Play?
Some resolution input images and settings produce mp4 files that video players don't like. You can reencode the video here if you need to. 

`-b:v 24000k is the bitrate.`



In [0]:
!ffmpeg -y -i ./videos/doublestrike.mp4 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" -c:v libx264 -crf 17 -b:v 24000k -pix_fmt yuv420p ./videos/doublestrike_fixed.mp4

Click below to play the generated video in the Colab Notebook. If you would like to re-run, start from the generator scripts above again and re-click this cell to update the video. You can also find your .mp4 file in the "Files" tab to the left.

In [0]:
# Show the generated video in this cell
# To download instead, right click the .mp4 file in the file manager on the left (the folder icon) and find the file, right click, download
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=500 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('./videos/TokyoHighRez.mp4')

# Old Stuff, didn't touch.

In [0]:
# Generate multiple videos from directory (old version)
#!python multi.py --fps 24
# Generate the autozoom video (original version)
#!python autozoom.py --fps '24' --in ./images/doublestrike.jpg